In [1]:
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import bigquery_storage
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder

In [2]:
key_path = '/home/jupyter/d00_key.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

job_config = bigquery.QueryJobConfig(allow_large_results=True)

#### Importing Training Data

In [3]:
Closed_store_data = pd.read_csv('/home/jupyter/HALO/closed_Store_weekly_phase_1.csv')
Closed_store_data.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,INSTORE_NEAR_A,INSTORE_NEAR_R,ONLINE_NEAR_N,ONLINE_NEAR_E,ONLINE_NEAR_A,ONLINE_NEAR_R,CENSUS_BLOCK_COUNTS,STORE_VOLUME_CD,FASHION_ATTRIBUTE_CD,OPENED_DAYS
0,5,1,355,378,40472.91,WINTER,84,84,9140.79,284,...,448,266,129,304,81,58,44,UNKNOWN,TF,15257
1,5,2,271,284,30694.04,WINTER,88,91,11119.08,213,...,334,199,95,229,58,43,44,UNKNOWN,TF,15257
2,5,3,279,287,31604.00,WINTER,77,81,8387.94,184,...,308,149,107,225,54,46,44,UNKNOWN,TF,15257
3,5,4,266,276,31157.94,WINTER,65,65,5660.71,193,...,300,191,106,216,51,39,44,UNKNOWN,TF,15257
4,5,5,264,273,28856.22,WINTER,62,65,7649.74,181,...,271,195,105,213,77,50,44,UNKNOWN,TF,15257


In [4]:
Closed_Factor = pd.read_csv('/home/jupyter/HALO/Closed_Stores_factor_Phase_1.csv')
Closed_Factor.rename(columns={'HALO Factor':'Closure Factor','store_number':'Store_Number'},inplace=True)
Closed_Factor.head()

,Store_Number,Closure Factor
0,5,-0.038678
1,41,-0.139055
2,84,-0.148988
3,89,-0.036485
4,91,-0.034105


#### Defaulting the store's positive HALO factor to be 0

In [5]:
Closed_Factor['Closure Factor'] = Closed_Factor['Closure Factor'].apply(lambda x:x if x<0 else 0)
Closed_Factor= Closed_Factor[Closed_Factor['Closure Factor']<0]
Closed_Factor.head()

,Store_Number,Closure Factor
0,5,-0.038678
1,41,-0.139055
2,84,-0.148988
3,89,-0.036485
4,91,-0.034105


In [6]:
Closed_Factor.shape

(53, 2)

In [7]:
Closed_store_data_v2 = Closed_store_data.merge(Closed_Factor, how='inner',left_on = 'STORE_NUMBER', right_on = 'Store_Number')
Closed_store_data_v2.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,ONLINE_NEAR_N,ONLINE_NEAR_E,ONLINE_NEAR_A,ONLINE_NEAR_R,CENSUS_BLOCK_COUNTS,STORE_VOLUME_CD,FASHION_ATTRIBUTE_CD,OPENED_DAYS,Store_Number,Closure Factor
0,5,1,355,378,40472.91,WINTER,84,84,9140.79,284,...,129,304,81,58,44,UNKNOWN,TF,15257,5,-0.038678
1,5,2,271,284,30694.04,WINTER,88,91,11119.08,213,...,95,229,58,43,44,UNKNOWN,TF,15257,5,-0.038678
2,5,3,279,287,31604.00,WINTER,77,81,8387.94,184,...,107,225,54,46,44,UNKNOWN,TF,15257,5,-0.038678
3,5,4,266,276,31157.94,WINTER,65,65,5660.71,193,...,106,216,51,39,44,UNKNOWN,TF,15257,5,-0.038678
4,5,5,264,273,28856.22,WINTER,62,65,7649.74,181,...,105,213,77,50,44,UNKNOWN,TF,15257,5,-0.038678


#### Creating the target (HALO Factor)*(ONLINE_GROSS_SALES) = STORE_ONLINE_GROSS_SALES (i.e Store's Online Contribution)

In [8]:
Closed_store_data_v2['STORE_ONLINE_GROSS_SALES'] = abs(Closed_store_data_v2['ONLINE_GROSS_SALES']*Closed_store_data_v2['Closure Factor'])
Closed_store_data_v2.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,ONLINE_NEAR_E,ONLINE_NEAR_A,ONLINE_NEAR_R,CENSUS_BLOCK_COUNTS,STORE_VOLUME_CD,FASHION_ATTRIBUTE_CD,OPENED_DAYS,Store_Number,Closure Factor,STORE_ONLINE_GROSS_SALES
0,5,1,355,378,40472.91,WINTER,84,84,9140.79,284,...,304,81,58,44,UNKNOWN,TF,15257,5,-0.038678,1565.408137
1,5,2,271,284,30694.04,WINTER,88,91,11119.08,213,...,229,58,43,44,UNKNOWN,TF,15257,5,-0.038678,1187.181746
2,5,3,279,287,31604.00,WINTER,77,81,8387.94,184,...,225,54,46,44,UNKNOWN,TF,15257,5,-0.038678,1222.377110
3,5,4,266,276,31157.94,WINTER,65,65,5660.71,193,...,216,51,39,44,UNKNOWN,TF,15257,5,-0.038678,1205.124435
4,5,5,264,273,28856.22,WINTER,62,65,7649.74,181,...,213,77,50,44,UNKNOWN,TF,15257,5,-0.038678,1116.098684


#### Importing public data to get the population information

In [9]:
key_path = '/home/jupyter/d00_key.json'
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

bq_client = bigquery.Client(credentials=credentials, project=credentials.project_id)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=credentials)

job_config = bigquery.QueryJobConfig(allow_large_results=True)

In [10]:
QUERY = """SELECT distinct geo_id, pop_25_years_over FROM `bigquery-public-data.census_bureau_acs.blockgroup_2018_5yr`; """

census_block_group = bq_client.query(QUERY, job_config=job_config).result().to_dataframe()
census_block_group['geo_id'] = census_block_group['geo_id'].apply(lambda x: str(x)[:-1])
census_block_group = census_block_group.groupby(['geo_id'])['pop_25_years_over'].sum().reset_index()
census_block_group.head()

,geo_id,pop_25_years_over
0,01001020100,1303.0
1,01001020200,1443.0
2,01001020300,2406.0
3,01001020400,2769.0
4,01001020500,6773.0


#### Importing closed store TA mapping

In [11]:
closed_store_clusters = pd.read_csv('/home/jupyter/HALO/Closed_Stores_BBBBY_store_clusters_BOPIS_Phase_1_no_HS.csv')
closed_store_clusters['FIPS_STATE_CD'] = closed_store_clusters['FIPS_STATE_CD'].apply(lambda x: str(x).zfill(2))
closed_store_clusters['FIPS_COUNTY_CD'] = closed_store_clusters['FIPS_COUNTY_CD'].apply(lambda x: str(x).zfill(3))
closed_store_clusters['CENSUS_BLOCK_CD'] = closed_store_clusters['CENSUS_BLOCK_CD'].apply(lambda x: str(x).zfill(6))


closed_store_clusters['GEO_ID'] = closed_store_clusters['FIPS_STATE_CD'] + closed_store_clusters['FIPS_COUNTY_CD'] + closed_store_clusters['CENSUS_BLOCK_CD']
closed_store_clusters.head()

,FIPS_STATE_CD,FIPS_COUNTY_CD,CENSUS_BLOCK_CD,GEOID,Store Number,GEO_ID
0,06,071,009104,6071009104,1004,06071009104
1,06,071,009105,6071009105,1004,06071009105
2,06,071,009107,6071009107,1004,06071009107
3,06,071,009108,6071009108,1004,06071009108
4,06,071,009109,6071009109,1004,06071009109


#### Importing open store TA mapping

In [12]:
store_clusters = pd.read_csv('/home/jupyter/HALO/BBBBY_store_clusters_BOPIS_Phase_1_no_HS.csv')
store_clusters['FIPS_STATE_CD'] = store_clusters['FIPS_STATE_CD'].apply(lambda x: str(x).zfill(2))
store_clusters['FIPS_COUNTY_CD'] = store_clusters['FIPS_COUNTY_CD'].apply(lambda x: str(x).zfill(3))
store_clusters['CENSUS_BLOCK_CD'] = store_clusters['CENSUS_BLOCK_CD'].apply(lambda x: str(x).zfill(6))


store_clusters['GEO_ID'] = store_clusters['FIPS_STATE_CD'] + store_clusters['FIPS_COUNTY_CD'] + store_clusters['CENSUS_BLOCK_CD']
store_clusters.head()

,FIPS_STATE_CD,FIPS_COUNTY_CD,CENSUS_BLOCK_CD,GEOID,Store Number,GEO_ID
0,48,039,660100,48039660100,1001,48039660100
1,48,039,660200,48039660200,1001,48039660200
2,48,039,660300,48039660300,1001,48039660300
3,48,039,660400,48039660400,1001,48039660400
4,48,039,660500,48039660500,1001,48039660500


#### Finding the overlapping TA

In [13]:
total_stores = pd.concat([store_clusters,closed_store_clusters])
total_stores.head()


,FIPS_STATE_CD,FIPS_COUNTY_CD,CENSUS_BLOCK_CD,GEOID,Store Number,GEO_ID
0,48,039,660100,48039660100,1001,48039660100
1,48,039,660200,48039660200,1001,48039660200
2,48,039,660300,48039660300,1001,48039660300
3,48,039,660400,48039660400,1001,48039660400
4,48,039,660500,48039660500,1001,48039660500


In [14]:
overlaps = total_stores.groupby(['GEO_ID'])['Store Number'].nunique().reset_index()
overlap_list = list(overlaps[overlaps['Store Number'] > 1]['GEO_ID'].unique())

In [15]:
merged_data = closed_store_clusters.merge(census_block_group, how = 'left', left_on='GEO_ID', right_on='geo_id')

total_population = merged_data.groupby(['Store Number'])['pop_25_years_over'].sum().reset_index()
total_population.head()


,Store Number,pop_25_years_over
0,5,151706.0
1,41,203001.0
2,84,264088.0
3,89,221724.0
4,91,418617.0


In [16]:
overlapped_customers = merged_data[merged_data['GEO_ID'].isin(overlap_list)].groupby(['Store Number'])['pop_25_years_over'].sum().reset_index().rename(columns={'pop_25_years_over':'overlapped_customers'})
overlapped_customers.head()

,Store Number,overlapped_customers
0,5,108073.0
1,41,174396.0
2,84,264088.0
3,89,182581.0
4,91,302174.0


#### Creating overlapped customers as one of the variable in training data

In [17]:
Data_v2 = Closed_store_data_v2.merge(overlapped_customers, how='left', left_on = 'STORE_NUMBER', right_on = 'Store Number').merge(total_population, how='left', left_on = 'STORE_NUMBER', right_on = 'Store Number').drop(['Store Number_x','Store Number_y'], axis=1)
Data_v2.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,ONLINE_NEAR_R,CENSUS_BLOCK_COUNTS,STORE_VOLUME_CD,FASHION_ATTRIBUTE_CD,OPENED_DAYS,Store_Number,Closure Factor,STORE_ONLINE_GROSS_SALES,overlapped_customers,pop_25_years_over
0,5,1,355,378,40472.91,WINTER,84,84,9140.79,284,...,58,44,UNKNOWN,TF,15257,5,-0.038678,1565.408137,108073.0,151706.0
1,5,2,271,284,30694.04,WINTER,88,91,11119.08,213,...,43,44,UNKNOWN,TF,15257,5,-0.038678,1187.181746,108073.0,151706.0
2,5,3,279,287,31604.00,WINTER,77,81,8387.94,184,...,46,44,UNKNOWN,TF,15257,5,-0.038678,1222.377110,108073.0,151706.0
3,5,4,266,276,31157.94,WINTER,65,65,5660.71,193,...,39,44,UNKNOWN,TF,15257,5,-0.038678,1205.124435,108073.0,151706.0
4,5,5,264,273,28856.22,WINTER,62,65,7649.74,181,...,50,44,UNKNOWN,TF,15257,5,-0.038678,1116.098684,108073.0,151706.0


#### Importing Competitor information (No Competitor information for BABY stores)

In [19]:
COMPETITORS = pd.read_excel('/home/jupyter/HALO/Regression Model/BBBY_ComPetitor_file.xlsx')
COMPETITORS.head()

,STORE_NBR,LOCATION NAME,ADDRESS,CITY,STATE,ZIP,HG 0.5 mi,HG 1 mi,HG 2.5 mi,HG 5 mi,...,Tar 10 mi,Tar 20 mi,Tar 50 mi,Wal 0.5 mi,Wal 1 mi,Wal 2.5 mi,Wal 5 mi,Wal 10 mi,Wal 20 mi,Wal 50 mi
0,1,SPRINGFIELD,715 MORRIS AVE,SPRINGFIELD,NJ,7081,NaN,NaN,NaN,1.0,...,6.0,29.0,70.0,NaN,NaN,NaN,1.0,4.0,21.0,52
1,2,BAY HARBOR,365 Rockaway Turnpike Unit 40,LAWRENCE,NY,11559,NaN,NaN,NaN,1.0,...,5.0,30.0,73.0,NaN,NaN,1.0,1.0,2.0,9.0,43
2,3,HUNTINGTON STATION,340 WALT WHITMAN ROAD,HUNTINGTON STATION,NY,11746,NaN,NaN,1.0,2.0,...,8.0,17.0,64.0,NaN,NaN,NaN,NaN,5.0,11.0,36
3,5,DANBURY,13 SUGAR HOLLOW RD.,DANBURY,CT,6810,NaN,NaN,NaN,1.0,...,1.0,4.0,53.0,NaN,NaN,NaN,1.0,1.0,5.0,45
4,6,STAMFORD,2275 SUMMER STREET,STAMFORD,CT,6905,NaN,NaN,NaN,NaN,...,1.0,6.0,68.0,NaN,NaN,NaN,NaN,2.0,3.0,44


#### Merging the competitor information with training data

In [20]:
#MERGED_DATA=Data_v2  ###for BABY
MERGED_DATA = Data_v2.merge(COMPETITORS, how='inner', left_on='STORE_NUMBER', right_on = 'STORE_NBR')  ## For BBB
MERGED_DATA.fillna(0, inplace=True)
MERGED_DATA.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,Tar 10 mi,Tar 20 mi,Tar 50 mi,Wal 0.5 mi,Wal 1 mi,Wal 2.5 mi,Wal 5 mi,Wal 10 mi,Wal 20 mi,Wal 50 mi
0,5,1,355,378,40472.91,WINTER,84,84,9140.79,284,...,1.0,4.0,53.0,0.0,0.0,0.0,1.0,1.0,5.0,45
1,5,2,271,284,30694.04,WINTER,88,91,11119.08,213,...,1.0,4.0,53.0,0.0,0.0,0.0,1.0,1.0,5.0,45
2,5,3,279,287,31604.00,WINTER,77,81,8387.94,184,...,1.0,4.0,53.0,0.0,0.0,0.0,1.0,1.0,5.0,45
3,5,4,266,276,31157.94,WINTER,65,65,5660.71,193,...,1.0,4.0,53.0,0.0,0.0,0.0,1.0,1.0,5.0,45
4,5,5,264,273,28856.22,WINTER,62,65,7649.74,181,...,1.0,4.0,53.0,0.0,0.0,0.0,1.0,1.0,5.0,45


#### Creating variable for holiday indicator

In [21]:
hol = []
for i,r in MERGED_DATA.iterrows():
    if r['WEEKS'] in ([1,4,8,22,27,36,42,46,48,52]):
        hol.append(1)
    else:
        hol.append(0)
MERGED_DATA['Holiday_ind'] = hol

#### Converting categorical variables to codes

In [153]:
enc_1 = LabelEncoder().fit(list(set(list(MERGED_DATA['CALENDAR_SEASON_CD']))))
MERGED_DATA['CALENDAR_SEASON_CD_cat'] = enc_1.transform(MERGED_DATA['CALENDAR_SEASON_CD'])
                                
enc_2 = LabelEncoder().fit(list(set(list(MERGED_DATA['STORE_VOLUME_CD']))) + list(set(list(Open_store_data_v4['STORE_VOLUME_CD']))))
#enc_2 = LabelEncoder().fit(list(['UNKNOWN','A', 'A++', 'A+', 'NY', 'B+', 'D', 'NY+', 'B', 'C']))
MERGED_DATA['STORE_VOLUME_CD_cat'] = enc_2.transform(MERGED_DATA['STORE_VOLUME_CD'])

enc_3 = LabelEncoder().fit(list(set(list(MERGED_DATA['FASHION_ATTRIBUTE_CD'])))+list(set(list(Open_store_data_v4['FASHION_ATTRIBUTE_CD']))))
MERGED_DATA['FASHION_ATTRIBUTE_CD_cat'] = enc_3.transform(MERGED_DATA['FASHION_ATTRIBUTE_CD'])

#### Replacing blanks in the column names

In [154]:
MERGED_DATA.columns = MERGED_DATA.columns.str.replace(' ', '_')
MERGED_DATA.columns

Index(['STORE_NUMBER', 'WEEKS', 'ONLINE_CUSTOMER_COUNTS',
       'ONLINE_TRANSACTION_COUNTS', 'ONLINE_GROSS_SALES', 'CALENDAR_SEASON_CD',
       'INSTORE_CUSTOMER_COUNTS', 'INSTORE_TRANSACTION_COUNTS',
       'INSTORE_GROSS_SALES', 'ALL_NEAR_N', 'ALL_NEAR_E', 'ALL_NEAR_A',
       'ALL_NEAR_R', 'INSTORE_NEAR_N', 'INSTORE_NEAR_E', 'INSTORE_NEAR_A',
       'INSTORE_NEAR_R', 'ONLINE_NEAR_N', 'ONLINE_NEAR_E', 'ONLINE_NEAR_A',
       'ONLINE_NEAR_R', 'CENSUS_BLOCK_COUNTS', 'STORE_VOLUME_CD',
       'FASHION_ATTRIBUTE_CD', 'OPENED_DAYS', 'Store_Number', 'Closure_Factor',
       'STORE_ONLINE_GROSS_SALES', 'overlapped_customers', 'pop_25_years_over',
       'STORE_NBR', 'LOCATION_NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP',
       'HG_0.5_mi', 'HG_1_mi', 'HG_2.5_mi', 'HG_5_mi', 'HG_10_mi', 'HG_20_mi',
       'HG_50_mi', 'HS_0.5_mi', 'HS_1_mi', 'HS_2.5_mi', 'HS_5_mi', 'HS_10_mi',
       'HS_20_mi', 'HS_50_mi', 'JCP_0.5_mi', 'JCP_1_mi', 'JCP_2.5_mi',
       'JCP_5_mi', 'JCP_10_mi', 'JCP_20_mi', 'JC

In [156]:
MERGED_DATA['Mod_Closure_Factor'] = MERGED_DATA['Closure_Factor']

#### Identifying variables with high correlation

In [157]:
CORRELATION_MATRIX = MERGED_DATA.drop(["STORE_ONLINE_GROSS_SALES", "STORE_VOLUME_CD","CALENDAR_SEASON_CD","FASHION_ATTRIBUTE_CD",'LOCATION_NAME','ADDRESS','CITY','STATE','ZIP'],axis=1).apply(lambda x: x.corr(MERGED_DATA.STORE_ONLINE_GROSS_SALES)).reset_index().rename(columns={'index':'COLUMNS',0:'CORRELATION'})
#CORRELATION_MATRIX = MERGED_DATA.drop(["STORE_ONLINE_GROSS_SALES", "STORE_VOLUME_CD","CALENDAR_SEASON_CD","FASHION_ATTRIBUTE_CD"],axis=1).apply(lambda x: x.corr(MERGED_DATA.STORE_ONLINE_GROSS_SALES)).reset_index().rename(columns={'index':'COLUMNS',0:'CORRELATION'})
CORRELATION_MATRIX['MOD_CORRELATION'] = CORRELATION_MATRIX['CORRELATION'].abs()
CORRELATION_MATRIX = CORRELATION_MATRIX.sort_values(by=['MOD_CORRELATION'], ascending=False)
CORRELATION_MATRIX.head(40)

,COLUMNS,CORRELATION,MOD_CORRELATION
4,ONLINE_GROSS_SALES,0.526910,0.526910
3,ONLINE_TRANSACTION_COUNTS,0.503694,0.503694
16,ONLINE_NEAR_N,0.503394,0.503394
2,ONLINE_CUSTOMER_COUNTS,0.495499,0.495499
18,ONLINE_NEAR_A,0.469813,0.469813
19,ONLINE_NEAR_R,0.468221,0.468221
7,INSTORE_GROSS_SALES,0.449946,0.449946
8,ALL_NEAR_N,0.421941,0.421941
6,INSTORE_TRANSACTION_COUNTS,0.419503,0.419503
5,INSTORE_CUSTOMER_COUNTS,0.416509,0.416509


#### Selecting the top 40 variables with high correlation

In [158]:
MERGED_DATA_v2 = MERGED_DATA[MERGED_DATA.columns.intersection(list(CORRELATION_MATRIX['COLUMNS'][:40])+['WEEKS','STORE_NUMBER','STORE_ONLINE_GROSS_SALES','Closure_Factor','Mod_Closure_Factor'])]
#MERGED_DATA_v2 = MERGED_DATA[MERGED_DATA.columns.intersection(list(CORRELATION_MATRIX['COLUMNS'][:40])+['STORE_NUMBER','WEEKS','STORE_ONLINE_GROSS_SALES','Holiday_ind','Kol_10_mi','Tar_10_mi','HG_0.5_mi','Wal_10_mi','HS_20_mi','Mcy_5_mi','Tar_5_mi','JCP_5_mi'])]
#MERGED_DATA_v2 = MERGED_DATA[MERGED_DATA.columns.intersection(list(CORRELATION_MATRIX['COLUMNS'][:40])+['STORE_NUMBER','WEEKS','STORE_ONLINE_GROSS_SALES','Holiday_ind'])]
MERGED_DATA_v2.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,ALL_NEAR_E,...,Kol_10_mi,Kol_20_mi,Mcy_5_mi,Mcy_10_mi,Tar_0.5_mi,Tar_5_mi,Tar_10_mi,Wal_0.5_mi,Wal_10_mi,Mod_Closure_Factor
0,5,1,355,378,40472.91,84,84,9140.79,284,3450,...,3.0,8.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-0.038678
1,5,2,271,284,30694.04,88,91,11119.08,213,2906,...,3.0,8.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-0.038678
2,5,3,279,287,31604.00,77,81,8387.94,184,2682,...,3.0,8.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-0.038678
3,5,4,266,276,31157.94,65,65,5660.71,193,2567,...,3.0,8.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-0.038678
4,5,5,264,273,28856.22,62,65,7649.74,181,2641,...,3.0,8.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,-0.038678


#### Calculating the feature importance

In [159]:
from sklearn.ensemble import RandomForestRegressor
forest = RandomForestRegressor()

X = MERGED_DATA_v2.drop(['STORE_NUMBER','STORE_ONLINE_GROSS_SALES','Closure_Factor','Mod_Closure_Factor'],axis=1)
y = MERGED_DATA_v2[['STORE_ONLINE_GROSS_SALES']]

forest.fit(X, y)

importances = forest.feature_importances_

/opt/conda/envs/HALO/lib/python3.7/site-packages/ipykernel_launcher.py:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  import sys


In [160]:
col_list = list(MERGED_DATA_v2.columns)
col_list.remove('STORE_NUMBER')
#col_list.remove('Store_Number')
#col_list.remove('STORE_NBR')
col_list.remove('STORE_ONLINE_GROSS_SALES')
col_list.remove('Closure_Factor')
col_list.remove('Mod_Closure_Factor')
#col_list.remove('WEEKS')
#col_list.remove('ONLINE_VISITS')

In [161]:
feature_imp_var = pd.DataFrame()
feature_imp_var['Variables'] = col_list
feature_imp_var['Importance'] = list(importances)
feature_imp_var = feature_imp_var.sort_values(['Importance'],ascending=False)
feature_imp_var.head(20)

,Variables,Importance
2,ONLINE_TRANSACTION_COUNTS,0.152849
19,overlapped_customers,0.113482
3,ONLINE_GROSS_SALES,0.112365
15,ONLINE_NEAR_N,0.098109
20,pop_25_years_over,0.076409
37,Wal_10_mi,0.045071
23,HG_20_mi,0.035867
28,Kol_2.5_mi,0.035020
16,ONLINE_NEAR_E,0.028991
12,INSTORE_NEAR_E,0.027246


In [576]:
#feature_imp_var.to_csv("/home/jupyter/HALO/Regression Model/Feature_Importance.csv", index=False)
CORRELATION_MATRIX.to_csv("/home/jupyter/HALO/Regression Model/Correlation_Importance.csv", index=False)

In [162]:
list(feature_imp_var[:30]['Variables'])

['ONLINE_TRANSACTION_COUNTS',
 'overlapped_customers',
 'ONLINE_GROSS_SALES',
 'ONLINE_NEAR_N',
 'pop_25_years_over',
 'Wal_10_mi',
 'HG_20_mi',
 'Kol_2.5_mi',
 'ONLINE_NEAR_E',
 'INSTORE_NEAR_E',
 'ALL_NEAR_E',
 'ONLINE_CUSTOMER_COUNTS',
 'Tar_10_mi',
 'ONLINE_NEAR_R',
 'JCP_10_mi',
 'Kol_20_mi',
 'WEEKS',
 'Kol_10_mi',
 'Mcy_10_mi',
 'ALL_NEAR_R',
 'ALL_NEAR_A',
 'INSTORE_GROSS_SALES',
 'INSTORE_NEAR_A',
 'INSTORE_NEAR_N',
 'ONLINE_NEAR_A',
 'HG_5_mi',
 'ALL_NEAR_N',
 'Wal_0.5_mi',
 'INSTORE_NEAR_R',
 'INSTORE_CUSTOMER_COUNTS']

In [31]:
#MERGED_DATA_v2['STORE_NUMBER'] = MERGED_DATA_v2['STORE_NUMBER'].astype('int')
#MERGED_DATA_v2['WEEKS'] = MERGED_DATA_v2['WEEKS'].astype('int')

In [163]:
feature_imp_var[:30]['Variables']

2     ONLINE_TRANSACTION_COUNTS
19         overlapped_customers
3            ONLINE_GROSS_SALES
15                ONLINE_NEAR_N
20            pop_25_years_over
37                    Wal_10_mi
23                     HG_20_mi
28                   Kol_2.5_mi
16                ONLINE_NEAR_E
12               INSTORE_NEAR_E
8                    ALL_NEAR_E
1        ONLINE_CUSTOMER_COUNTS
35                    Tar_10_mi
18                ONLINE_NEAR_R
25                    JCP_10_mi
30                    Kol_20_mi
0                         WEEKS
29                    Kol_10_mi
32                    Mcy_10_mi
10                   ALL_NEAR_R
9                    ALL_NEAR_A
6           INSTORE_GROSS_SALES
13               INSTORE_NEAR_A
11               INSTORE_NEAR_N
17                ONLINE_NEAR_A
21                      HG_5_mi
7                    ALL_NEAR_N
36                   Wal_0.5_mi
14               INSTORE_NEAR_R
4       INSTORE_CUSTOMER_COUNTS
Name: Variables, dtype: object

#### Importing Scoring Data

In [172]:
Open_store_data = pd.read_csv('/home/jupyter/HALO/BBBY_Store_weekly_phase_1.csv')
Open_store_data.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,INSTORE_NEAR_A,INSTORE_NEAR_R,ONLINE_NEAR_N,ONLINE_NEAR_E,ONLINE_NEAR_A,ONLINE_NEAR_R,CENSUS_BLOCK_COUNTS,STORE_VOLUME_CD,FASHION_ATTRIBUTE_CD,OPENED_DAYS
0,1,1,686,749,90715.43,WINTER,162,169,16035.77,397,...,638,430,183,590,138,118,61,A,C,18390
1,1,2,627,671,74808.09,WINTER,170,178,20986.30,368,...,556,383,206,459,127,91,61,A,C,18390
2,1,3,559,621,78338.63,WINTER,159,169,18364.58,338,...,480,286,191,477,107,82,61,A,C,18390
3,1,4,582,662,79914.92,WINTER,154,164,19765.32,326,...,489,322,197,473,121,69,61,A,C,18390
4,1,5,607,665,80429.29,WINTER,128,132,15681.80,324,...,472,252,199,466,136,113,61,A,C,18390


#### Importing BBBY open store clusters

In [175]:
open_store_clusters = pd.read_csv('/home/jupyter/HALO/BBBBY_store_clusters_BOPIS_Phase_1_no_HS.csv')
open_store_clusters['FIPS_STATE_CD'] = open_store_clusters['FIPS_STATE_CD'].apply(lambda x: str(x).zfill(2))
open_store_clusters['FIPS_COUNTY_CD'] = open_store_clusters['FIPS_COUNTY_CD'].apply(lambda x: str(x).zfill(3))
open_store_clusters['CENSUS_BLOCK_CD'] = open_store_clusters['CENSUS_BLOCK_CD'].apply(lambda x: str(x).zfill(6))


open_store_clusters['GEO_ID'] = open_store_clusters['FIPS_STATE_CD'] + open_store_clusters['FIPS_COUNTY_CD'] + open_store_clusters['CENSUS_BLOCK_CD']
open_store_clusters.head()

,FIPS_STATE_CD,FIPS_COUNTY_CD,CENSUS_BLOCK_CD,GEOID,Store Number,GEO_ID
0,48,039,660100,48039660100,1001,48039660100
1,48,039,660200,48039660200,1001,48039660200
2,48,039,660300,48039660300,1001,48039660300
3,48,039,660400,48039660400,1001,48039660400
4,48,039,660500,48039660500,1001,48039660500


#### Creating overlapping census tract list

In [176]:
overlaps = open_store_clusters.groupby(['GEO_ID'])['Store Number'].nunique().reset_index()
overlap_list = list(overlaps[overlaps['Store Number'] > 1]['GEO_ID'].unique())

In [177]:
open_merged_data = open_store_clusters.merge(census_block_group, how = 'left', left_on='GEO_ID', right_on='geo_id')

open_total_population = open_merged_data.groupby(['Store Number'])['pop_25_years_over'].sum().reset_index()
open_total_population.head()


,Store Number,pop_25_years_over
0,1,202927.0
1,6,174807.0
2,8,397458.0
3,9,235056.0
4,12,138949.0


In [179]:
open_overlapped_customers = open_merged_data[open_merged_data['GEO_ID'].isin(overlap_list)].groupby(['Store Number'])['pop_25_years_over'].sum().reset_index().rename(columns={'pop_25_years_over':'overlapped_customers'})
open_overlapped_customers.head()

,Store Number,overlapped_customers
0,1,147352.0
1,6,76687.0
2,8,21380.0
3,9,151773.0
4,12,54458.0


#### Creating overlapped variable in the scoring data

In [180]:
Open_store_data_v3 = Open_store_data.merge(open_overlapped_customers, how='left', left_on = 'STORE_NUMBER', right_on = 'Store Number').merge(open_total_population, how='left', left_on = 'STORE_NUMBER', right_on = 'Store Number').drop(['Store Number_x','Store Number_y'], axis=1)
Open_store_data_v3.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,ONLINE_NEAR_N,ONLINE_NEAR_E,ONLINE_NEAR_A,ONLINE_NEAR_R,CENSUS_BLOCK_COUNTS,STORE_VOLUME_CD,FASHION_ATTRIBUTE_CD,OPENED_DAYS,overlapped_customers,pop_25_years_over
0,1,1,686,749,90715.43,WINTER,162,169,16035.77,397,...,183,590,138,118,61,A,C,18390,147352.0,202927.0
1,1,2,627,671,74808.09,WINTER,170,178,20986.30,368,...,206,459,127,91,61,A,C,18390,147352.0,202927.0
2,1,3,559,621,78338.63,WINTER,159,169,18364.58,338,...,191,477,107,82,61,A,C,18390,147352.0,202927.0
3,1,4,582,662,79914.92,WINTER,154,164,19765.32,326,...,197,473,121,69,61,A,C,18390,147352.0,202927.0
4,1,5,607,665,80429.29,WINTER,128,132,15681.80,324,...,199,466,136,113,61,A,C,18390,147352.0,202927.0


#### Joining the competitor information with scoring data

In [181]:
Open_store_data_v4 = Open_store_data_v3.merge(COMPETITORS, how='left', left_on='STORE_NUMBER', right_on = 'STORE_NBR')
#Open_store_data_v4 = Open_store_data_v3
Open_store_data_v4.fillna(0, inplace=True)
Open_store_data_v4.head()

,STORE_NUMBER,WEEKS,ONLINE_CUSTOMER_COUNTS,ONLINE_TRANSACTION_COUNTS,ONLINE_GROSS_SALES,CALENDAR_SEASON_CD,INSTORE_CUSTOMER_COUNTS,INSTORE_TRANSACTION_COUNTS,INSTORE_GROSS_SALES,ALL_NEAR_N,...,Tar 10 mi,Tar 20 mi,Tar 50 mi,Wal 0.5 mi,Wal 1 mi,Wal 2.5 mi,Wal 5 mi,Wal 10 mi,Wal 20 mi,Wal 50 mi
0,1,1,686,749,90715.43,WINTER,162,169,16035.77,397,...,6.0,29.0,70.0,0.0,0.0,0.0,1.0,4.0,21.0,52.0
1,1,2,627,671,74808.09,WINTER,170,178,20986.30,368,...,6.0,29.0,70.0,0.0,0.0,0.0,1.0,4.0,21.0,52.0
2,1,3,559,621,78338.63,WINTER,159,169,18364.58,338,...,6.0,29.0,70.0,0.0,0.0,0.0,1.0,4.0,21.0,52.0
3,1,4,582,662,79914.92,WINTER,154,164,19765.32,326,...,6.0,29.0,70.0,0.0,0.0,0.0,1.0,4.0,21.0,52.0
4,1,5,607,665,80429.29,WINTER,128,132,15681.80,324,...,6.0,29.0,70.0,0.0,0.0,0.0,1.0,4.0,21.0,52.0


In [182]:
Open_store_data_v4.columns = Open_store_data_v4.columns.str.replace(' ', '_')

#### Converting categorical information into codes

In [185]:
Open_store_data_v4['CALENDAR_SEASON_CD_cat'] = enc_1.transform(Open_store_data_v4['CALENDAR_SEASON_CD'])
Open_store_data_v4['STORE_VOLUME_CD_cat'] = enc_2.transform(Open_store_data_v4['STORE_VOLUME_CD'])
Open_store_data_v4['FASHION_ATTRIBUTE_CD_cat'] = enc_3.transform(Open_store_data_v4['FASHION_ATTRIBUTE_CD'])

#### Creating holiday indicator

In [186]:
hol = []
for i,r in Open_store_data_v4.iterrows():
    if r['WEEKS'] in ([1,4,8,22,27,36,42,46,48,52]):
        hol.append(1)
    else:
        hol.append(0)
Open_store_data_v4['Holiday_ind'] = hol

#### Selecting necessary columns in Scoring Data

In [188]:
X_TEST = Open_store_data_v4[['ONLINE_TRANSACTION_COUNTS',
 'overlapped_customers',
 'ONLINE_GROSS_SALES',
 'ONLINE_NEAR_N',
 'pop_25_years_over',
 'Wal_10_mi',
 'HG_20_mi',
 'Kol_2.5_mi',
 'ONLINE_NEAR_E',
 'INSTORE_NEAR_E',
 'ALL_NEAR_E',
 'ONLINE_CUSTOMER_COUNTS',
 'Tar_10_mi',
 'ONLINE_NEAR_R',
 'JCP_10_mi',
 'Kol_20_mi',
 'WEEKS',
 'Kol_10_mi',
 'Mcy_10_mi',
 'ALL_NEAR_R',
 'ALL_NEAR_A',
 'INSTORE_GROSS_SALES',
 'INSTORE_NEAR_A',
 'INSTORE_NEAR_N',
 'ONLINE_NEAR_A',
 'HG_5_mi',
 'ALL_NEAR_N',
 'Wal_0.5_mi',
 'INSTORE_NEAR_R',
 'INSTORE_CUSTOMER_COUNTS']]

X_TEST

,ONLINE_TRANSACTION_COUNTS,overlapped_customers,ONLINE_GROSS_SALES,ONLINE_NEAR_N,pop_25_years_over,Wal_10_mi,HG_20_mi,Kol_2.5_mi,ONLINE_NEAR_E,INSTORE_NEAR_E,...,ALL_NEAR_A,INSTORE_GROSS_SALES,INSTORE_NEAR_A,INSTORE_NEAR_N,ONLINE_NEAR_A,HG_5_mi,ALL_NEAR_N,Wal_0.5_mi,INSTORE_NEAR_R,INSTORE_CUSTOMER_COUNTS
0,749,147352.0,90715.43,183,202927.0,4.0,24.0,0.0,590,4053,...,696,16035.77,638,389,138,1.0,397,0.0,430,162
1,671,147352.0,74808.09,206,202927.0,4.0,24.0,0.0,459,3532,...,592,20986.30,556,352,127,1.0,368,0.0,383,170
2,621,147352.0,78338.63,191,202927.0,4.0,24.0,0.0,477,3351,...,500,18364.58,480,330,107,1.0,338,0.0,286,159
3,662,147352.0,79914.92,197,202927.0,4.0,24.0,0.0,473,3032,...,530,19765.32,489,295,121,1.0,326,0.0,322,154
4,665,147352.0,80429.29,199,202927.0,4.0,24.0,0.0,466,2977,...,503,15681.80,472,303,136,1.0,324,0.0,252,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39214,1202,68325.0,160401.27,292,119212.0,0.0,0.0,0.0,617,2468,...,479,14323.02,377,314,196,0.0,384,0.0,232,102
39215,900,68325.0,116725.35,233,119212.0,0.0,0.0,0.0,513,2281,...,552,8377.01,488,373,140,0.0,417,0.0,280,54
39216,705,68325.0,74820.76,165,119212.0,0.0,0.0,0.0,453,2387,...,518,7323.56,485,327,127,0.0,336,0.0,277,49
39217,591,68325.0,59587.56,150,119212.0,0.0,0.0,0.0,483,2903,...,618,6116.77,576,445,110,0.0,437,0.0,332,54


#### Oversampling training data

In [189]:
import smogn


training = MERGED_DATA_v2[['ONLINE_TRANSACTION_COUNTS',
 'overlapped_customers',
 'ONLINE_GROSS_SALES',
 'ONLINE_NEAR_N',
 'pop_25_years_over',
 'Wal_10_mi',
 'HG_20_mi',
 'Kol_2.5_mi',
 'ONLINE_NEAR_E',
 'INSTORE_NEAR_E',
 'ALL_NEAR_E',
 'ONLINE_CUSTOMER_COUNTS',
 'Tar_10_mi',
 'ONLINE_NEAR_R',
 'JCP_10_mi',
 'Kol_20_mi',
 'WEEKS',
 'Kol_10_mi',
 'Mcy_10_mi',
 'ALL_NEAR_R',
 'ALL_NEAR_A',
 'INSTORE_GROSS_SALES',
 'INSTORE_NEAR_A',
 'INSTORE_NEAR_N',
 'ONLINE_NEAR_A',
 'HG_5_mi',
 'ALL_NEAR_N',
 'Wal_0.5_mi',
 'INSTORE_NEAR_R',
 'INSTORE_CUSTOMER_COUNTS',
 'STORE_ONLINE_GROSS_SALES']]


MERGED_DATA_v2_smogn = smogn.smoter(
    data = training, 
    y = "STORE_ONLINE_GROSS_SALES",
    k = 3,  
  samp_method = 'extreme',
  rel_thres = 0.80,
  rel_method = 'auto',
  rel_xtrm_type = 'both',
 rel_coef = 2.5
    )


#X = MERGED_DATA_v2_smogn[list(feature_imp_var[:30]['Variables'])]

X = MERGED_DATA_v2_smogn[['ONLINE_TRANSACTION_COUNTS',
 'overlapped_customers',
 'ONLINE_GROSS_SALES',
 'ONLINE_NEAR_N',
 'pop_25_years_over',
 'Wal_10_mi',
 'HG_20_mi',
 'Kol_2.5_mi',
 'ONLINE_NEAR_E',
 'INSTORE_NEAR_E',
 'ALL_NEAR_E',
 'ONLINE_CUSTOMER_COUNTS',
 'Tar_10_mi',
 'ONLINE_NEAR_R',
 'JCP_10_mi',
 'Kol_20_mi',
 'WEEKS',
 'Kol_10_mi',
 'Mcy_10_mi',
 'ALL_NEAR_R',
 'ALL_NEAR_A',
 'INSTORE_GROSS_SALES',
 'INSTORE_NEAR_A',
 'INSTORE_NEAR_N',
 'ONLINE_NEAR_A',
 'HG_5_mi',
 'ALL_NEAR_N',
 'Wal_0.5_mi',
 'INSTORE_NEAR_R',
 'INSTORE_CUSTOMER_COUNTS']]
Y = MERGED_DATA_v2_smogn['STORE_ONLINE_GROSS_SALES']

X

r_index: 100%|##########| 29/29 [00:00<00:00, 368.56it/s]


,ONLINE_TRANSACTION_COUNTS,overlapped_customers,ONLINE_GROSS_SALES,ONLINE_NEAR_N,pop_25_years_over,Wal_10_mi,HG_20_mi,Kol_2.5_mi,ONLINE_NEAR_E,INSTORE_NEAR_E,...,ALL_NEAR_A,INSTORE_GROSS_SALES,INSTORE_NEAR_A,INSTORE_NEAR_N,ONLINE_NEAR_A,HG_5_mi,ALL_NEAR_N,Wal_0.5_mi,INSTORE_NEAR_R,INSTORE_CUSTOMER_COUNTS
0,731,255747.537407,80173.618334,289,289707.780053,6.740584,4.0,0.0,250,1860,...,700,9274.333847,594,497,132,0.259416,577,0.0,425,80
1,652,264088.000000,66954.030843,247,264088.000000,6.000000,4.0,0.0,329,2872,...,1067,14255.945853,984,813,125,1.000000,813,0.0,759,96
2,615,263172.465429,66618.446812,235,266900.289375,6.081294,4.0,0.0,311,2756,...,1030,13774.572061,950,790,116,0.918706,789,0.0,734,93
3,696,264088.000000,71552.965539,261,264088.000000,6.000000,4.0,0.0,324,2682,...,944,12331.357702,853,686,143,1.000000,706,0.0,652,94
4,610,263000.111028,66712.976820,233,267429.718265,6.096598,4.0,0.0,307,2727,...,1019,13617.953301,939,781,115,0.903402,781,0.0,725,92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,847,252826.000000,97181.990000,353,298682.000000,7.000000,4.0,0.0,224,1601,...,679,12725.420000,529,503,184,0.000000,624,0.0,373,95
2752,762,252826.000000,85097.680000,305,298682.000000,7.000000,4.0,0.0,223,1494,...,564,7406.550000,449,379,136,0.000000,488,0.0,302,75
2753,590,252826.000000,50160.110000,254,298682.000000,7.000000,4.0,0.0,188,1598,...,647,6703.850000,590,498,104,0.000000,574,0.0,408,77
2754,421,252826.000000,44395.990000,211,298682.000000,7.000000,4.0,0.0,153,1982,...,843,5359.870000,774,810,73,0.000000,839,0.0,654,60


#### Creating Regression model

In [190]:
forest = RandomForestRegressor(n_estimators= 37,
 min_samples_split= 2,
min_samples_leaf= 1,
max_features= 'sqrt',
max_depth=None,
bootstrap= False)
forest.fit(X, Y)

#### Predicting for the scoring data

In [ ]:
open_stores_predictions = forest.predict(X_TEST)

#### Calculating the HALO factor (Forecasts/actual online gross sales)

In [ ]:
Open_store_data_v4['FORECASTS']=list(open_stores_predictions)
Open_store_data_v4['CLOSURE_FACTOR'] = (Open_store_data_v4['FORECASTS']/Open_store_data_v4['ONLINE_GROSS_SALES'])*100

#### Excluding the holiday weeks and weeks that are subject to high seasonality

In [191]:
Open_store_data_v5 = Open_store_data_v4[~Open_store_data_v4['WEEKS'].isin([1,4,8,22,27,36,42,46,48,52])]

Open_store_data_v5 = Open_store_data_v5[Open_store_data_v5['WEEKS']<=40]

Open_store_data_v5[['STORE_NUMBER','WEEKS','ONLINE_GROSS_SALES','FORECASTS','CLOSURE_FACTOR']]

,STORE_NUMBER,WEEKS,ONLINE_GROSS_SALES,FORECASTS,CLOSURE_FACTOR
1,1,2,74808.09,6087.747421,8.137820
2,1,3,78338.63,5579.111383,7.121788
4,1,5,80429.29,5961.665023,7.412306
5,1,6,71831.12,6474.133753,9.012993
6,1,7,66874.39,6915.273875,10.340691
...,...,...,...,...,...
39201,1901,35,59360.02,5281.206104,8.896908
39203,1901,37,60062.87,6644.448741,11.062490
39204,1901,38,61041.90,5925.040408,9.706514
39205,1901,39,45399.45,5015.476266,11.047438


#### Aggregating the closure factor at store level

In [192]:
Open_Store_Closure_Factor = Open_store_data_v5.groupby(['STORE_NUMBER'])['CLOSURE_FACTOR'].mean().reset_index()
Open_Store_Closure_Factor.head()

,STORE_NUMBER,CLOSURE_FACTOR
0,1,8.356227
1,6,7.876295
2,8,9.591047
3,9,6.572834
4,12,7.806381


In [193]:
# Mean closure factor
Open_Store_Closure_Factor['CLOSURE_FACTOR'].mean()

17.483143095817248

In [195]:
Open_Store_Closure_Factor.sort_values(['CLOSURE_FACTOR'],ascending=False,inplace=True)
Open_Store_Closure_Factor.head(10)

,STORE_NUMBER,CLOSURE_FACTOR
406,577,1371.590575
151,231,99.679770
233,356,72.307819
681,1278,56.781034
185,288,47.897763
223,338,47.121659
446,760,45.310900
751,1436,44.006959
310,457,42.762385
374,535,42.297119


In [196]:
# Identifying Outliers
Open_Store_Closure_Factor[Open_Store_Closure_Factor['CLOSURE_FACTOR']>=100]

,STORE_NUMBER,CLOSURE_FACTOR
406,577,1371.590575


In [197]:
# Mean closure factor after removing outliers
Open_Store_Closure_Factor[Open_Store_Closure_Factor['CLOSURE_FACTOR']<100]['CLOSURE_FACTOR'].mean()

15.687244645534472

#### Exporting the scored aggregated data

In [198]:
Open_Store_Closure_Factor.to_csv("/home/jupyter/HALO/Forecasted_Open_Store_Closure_Factor_BBBY_Phase_1.csv",index=False)

In [229]:
Open_store_data_v5[['STORE_NUMBER','pop_25_years_over','CENSUS_BLOCK_COUNTS']]

,STORE_NUMBER,pop_25_years_over,CENSUS_BLOCK_COUNTS
1,1,0.0,66
2,1,0.0,66
4,1,0.0,66
5,1,0.0,66
6,1,0.0,66
...,...,...,...
39238,1901,0.0,51
39240,1901,0.0,51
39241,1901,0.0,51
39242,1901,0.0,51


In [199]:
Open_store_data_v4[['STORE_NUMBER','pop_25_years_over','CENSUS_BLOCK_COUNTS']].drop_duplicates().to_csv("/home/jupyter/HALO/BBBY_open_store_info_Phase_1.csv",index=False)

### Development work

#### Tuning the Hyper parameters

In [107]:
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 30, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)
{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

{'n_estimators': [30, 37, 45, 53, 61, 68, 76, 84, 92, 100], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


{'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [109]:
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X, Y)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [30, 37, 45, 53, 61, 68,
                                                         76, 84, 92, 100]},
                   random_state=42, verbose=2)

In [110]:
rf_random.best_params_

{'n_estimators': 37,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}